In [ ]:
import math, time, os, sys, gc, random
import numpy as np
import pandas as pd
from collections import Counter
from itertools import product

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset

Configs

In [ ]:
RANDOM_STATE = 42
def set_seeds(seed=RANDOM_STATE):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seeds()

OUTER_K = 10
INNER_K = 3
SCORING  = "neg_rmse"   # inner-loop selection: maximize negative mean RMSE
BATCH_SIZE = 1024
MAX_EPOCHS = 30
PATIENCE   = 5
VERBOSE = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


Metrics Helpers

In [ ]:
def rmse(a,b): return float(np.sqrt(np.mean((a-b)**2)))
def mae(a,b):  return float(np.mean(np.abs(a-b)))
def r2(a,b):
    ssr = np.sum((a-b)**2); sst = np.sum((a - np.mean(a))**2)
    return float(1.0 - ssr / (sst + 1e-12))

Dataset Preparation

In [ ]:
df = pd.read_csv("/content/NEWUS_Accidents_Featured_Engineered.csv")

TARGETS = ["Severity_count", "Severity_mean", "Severity_max"]
for t in TARGETS:
    assert t in df.columns, f"df must contain target '{t}'"
# We'll stratify folds by risk_level if present; else fall back to regular KFold
HAS_RISK_LEVEL = "risk_level" in df.columns

# Forbid leakage / IDs / targets in features
FORBID = set(["grid_id","Severity","duration_minutes","Weather_Lag_Mins"] + TARGETS + (["risk_level"] if HAS_RISK_LEVEL else [])) & set(df.columns)


Feature Grouping

In [ ]:
def detect_groups(df: pd.DataFrame):
    cols = set(df.columns)
    weather_cols = [c for c in [
        'Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)',
        'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Has_Precipitation'
    ] if c in cols and c not in FORBID]

    time_cols = [c for c in [
        'Start_Hour_sin','Start_Hour_cos','Start_DayOfWeek_sin','Start_DayOfWeek_cos',
        'Start_Month_sin','Start_Month_cos','Sunrise_Sunset_binary','Civil_Twilight_binary',
        'Nautical_Twilight_binary','Astronomical_Twilight_binary','is_Weekend',
        'is_RushHour_Morning','is_RushHour_Evening','is_Winter','Wind_Direction_sin','Wind_Direction_cos'
    ] if c in cols and c not in FORBID]

    poi_cols = [c for c in [
        'Amenity','Bump','Crossing','Give_Way','Junction','No_Exit','Railway','Roundabout','Station','Stop',
        'Traffic_Calming','Traffic_Signal','Turning_Loop'
    ] if c in cols and c not in FORBID]

    emb_cols = [c for c in df.columns if c.startswith("Emb_PCA_") and c not in FORBID]
    city_oh  = [c for c in df.columns if c.startswith("City_")   and c not in FORBID]
    county_oh= [c for c in df.columns if c.startswith("County_") and c not in FORBID]
    wcat_oh  = [c for c in df.columns if c.startswith("Weather_") and (not c.startswith("Emb_")) and c not in FORBID]
    return {"weather":weather_cols,"time":time_cols,"poi":poi_cols,"embed":emb_cols,
            "city_oh":city_oh,"county_oh":county_oh,"wcat_oh":wcat_oh}

GROUPS = detect_groups(df)
print("Group sizes:", {k: len(v) for k,v in GROUPS.items()})

Group sizes: {'weather': 9, 'time': 16, 'poi': 13, 'embed': 50, 'city_oh': 30, 'county_oh': 30, 'wcat_oh': 8}


Manual cross-validation

In [ ]:
def stratified_kfold_indices(y, n_splits=5, random_state=42):
    rng = np.random.RandomState(random_state)
    y = np.asarray(y)
    classes, y_idx = np.unique(y, return_inverse=True)
    per_class = []
    for c in range(len(classes)):
        idx = np.where(y_idx == c)[0]; rng.shuffle(idx); per_class.append(idx)
    folds = [[] for _ in range(n_splits)]
    for arr in per_class:
        for i, j in enumerate(arr):
            folds[i % n_splits].append(j)
    all_idx = np.arange(len(y))
    pairs = []
    for k in range(n_splits):
        te = np.array(sorted(folds[k]), dtype=int)
        tr = np.setdiff1d(all_idx, te, assume_unique=False)
        pairs.append((tr, te))
    return pairs

def kfold_indices(n, n_splits=5, random_state=42):
    rng = np.random.RandomState(random_state)
    idx = np.arange(n); rng.shuffle(idx)
    folds = np.array_split(idx, n_splits)
    pairs = []
    all_idx = np.arange(n)
    for k in range(n_splits):
        te = np.array(sorted(folds[k]), dtype=int)
        tr = np.setdiff1d(all_idx, te, assume_unique=False)
        pairs.append((tr, te))
    return pairs

Define Blocks

In [ ]:
def onehot_to_id(mat_oh):
    if mat_oh.size == 0: return np.zeros((mat_oh.shape[0],), dtype=np.int64)
    argmax = mat_oh.argmax(axis=1) + 1
    has_any = (mat_oh.sum(axis=1) > 0).astype(np.int64)
    return argmax * has_any

def fit_medians(X):
    med = np.nanmedian(X, axis=0); med = np.where(np.isnan(med), 0.0, med); return med
def apply_medians(X, med): return np.where(np.isnan(X), med, X)

def build_numpy_blocks_strict(df_part: pd.DataFrame, groups):
    # 1) CONCAT all continuous columns -> one numeric matrix
    all_num_cols = groups["weather"] + groups["time"] + groups["poi"] + groups["embed"]
    X_num = df_part[all_num_cols].astype(float).values if len(all_num_cols) else np.zeros((len(df_part),0),dtype=np.float32)

    # 2) One-hot groups -> IDs (0 = none/other)
    out = {"num": X_num, "num_cols": all_num_cols}
    for block in ["city_oh","county_oh","wcat_oh"]:
        cols = groups[block]
        if len(cols):
            mat = df_part[cols].astype(float).values
            out[block.replace("_oh","_id")] = onehot_to_id(mat)
            out[block+"_K"] = len(cols)
        else:
            out[block.replace("_oh","_id")] = np.zeros((len(df_part),), dtype=np.int64)
            out[block+"_K"] = 0
    return out


Dataset

In [ ]:
class TabDataset(Dataset):
    def __init__(self, blocks, y_norm):
        self.blocks = blocks
        self.y_norm = y_norm
    def __len__(self): return len(self.y_norm)
    def __getitem__(self, i):
        sample = {
            "num": self.blocks["num"][i],                 # (n_num,)
            "city_id":   int(self.blocks["city_id"][i]),
            "county_id": int(self.blocks["county_id"][i]),
            "wcat_id":   int(self.blocks["wcat_id"][i]),
        }
        return sample, self.y_norm[i]

def collate_fn(batch):
    Xs, ys = zip(*batch)
    num      = torch.tensor(np.stack([x["num"] for x in Xs], 0), dtype=torch.float32)       # (B, n_num)
    city_id   = torch.tensor([x["city_id"]   for x in Xs], dtype=torch.long)
    county_id = torch.tensor([x["county_id"] for x in Xs], dtype=torch.long)
    wcat_id   = torch.tensor([x["wcat_id"]   for x in Xs], dtype=torch.long)
    y = torch.tensor(np.stack(ys, 0), dtype=torch.float32)
    return {"num": num, "city_id": city_id, "county_id": county_id, "wcat_id": wcat_id}, y

class NumFeatureTokenizer(nn.Module):
    """
    Strict FT-Transformer tokenizer: one token per numeric feature.
    token_j = x_j * w_j + b_j   (broadcasted over batch)
    """
    def __init__(self, n_num, d_token):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(n_num, d_token) * 0.02)
        self.bias   = nn.Parameter(torch.zeros(n_num, d_token))
    def forward(self, x_num):                 # x_num: (B, n_num)
        return x_num.unsqueeze(-1) * self.weight + self.bias   # (B, n_num, d_token)


FT-Transformer

In [ ]:
class FTTransformer(nn.Module):
    def __init__(self,
                 n_num,                 # number of numeric features
                 d_token=160, n_layers=2, n_heads=8, dropout=0.1,
                 K_city=0, K_county=0, K_wcat=0,
                 n_outputs=3):
        super().__init__()
        self.d = d_token

        # Strict tokenizer: one token per numeric feature
        self.num_tok = NumFeatureTokenizer(n_num, d_token)

        # Categorical embeddings (+1 for 'none')
        self.emb_city   = nn.Embedding(K_city+1,   d_token) if K_city>0 else None
        self.emb_county = nn.Embedding(K_county+1, d_token) if K_county>0 else None
        self.emb_wcat   = nn.Embedding(K_wcat+1,   d_token) if K_wcat>0 else None

        # CLS token
        self.cls = nn.Parameter(torch.zeros(1, 1, d_token))
        nn.init.trunc_normal_(self.cls, std=0.02)

        # Encoder
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_token, nhead=n_heads, dim_feedforward=4*d_token,
            dropout=dropout, batch_first=True, activation='gelu', norm_first=True
        )
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)

        # Head
        self.head = nn.Sequential(
            nn.LayerNorm(d_token),
            nn.Linear(d_token, d_token),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_token, n_outputs)
        )

    def forward(self, xb):
        # numeric features -> per-feature tokens
        tok_num = self.num_tok(xb["num"])           # (B, n_num, d)

        tokens = [tok_num]                          # list of (B, T_i, d)
        if self.emb_city   is not None: tokens.append(self.emb_city(xb["city_id"]).unsqueeze(1))     # (B,1,d)
        if self.emb_county is not None: tokens.append(self.emb_county(xb["county_id"]).unsqueeze(1)) # (B,1,d)
        if self.emb_wcat   is not None: tokens.append(self.emb_wcat(xb["wcat_id"]).unsqueeze(1))     # (B,1,d)

        x = torch.cat(tokens, dim=1)                # (B, T_total, d)
        cls = self.cls.expand(x.size(0), 1, self.d)
        x = torch.cat([cls, x], dim=1)              # prepend CLS
        x = self.encoder(x)
        cls_out = x[:, 0, :]
        y_hat = self.head(cls_out)                  # (B, 3)
        return y_hat


Train and Evaluation

In [ ]:
def train_one(model, loader, optimizer, criterion):
    model.train(); total = 0.0
    for xb, yb in loader:
        for k in xb: xb[k] = xb[k].to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)                 # (B,3) normalized targets
        optimizer.zero_grad(set_to_none=True)
        pred = model(xb)                                      # (B,3) normalized preds
        loss = criterion(pred, yb)
        loss.backward(); optimizer.step()
        total += loss.item() * yb.size(0)
    return total / len(loader.dataset)

@torch.no_grad()
def eval_one_norm(model, loader):
    model.eval()
    preds, trues = [], []
    for xb, yb in loader:
        for k in xb: xb[k] = xb[k].to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        out = model(xb)        # normalized
        preds.append(out.detach().cpu().numpy())
        trues.append(yb.detach().cpu().numpy())
    y_pred_norm = np.vstack(preds)   # (N,3)
    y_true_norm = np.vstack(trues)   # (N,3)
    return y_true_norm, y_pred_norm

# Utilities to (de)normalize + log transforms for count
def apply_log1p_count(y):      # y: (N,3) in ORIGINAL space
    y2 = y.copy()
    y2[:,0] = np.log1p(np.clip(y2[:,0], a_min=0, a_max=None))
    return y2

def invert_log1p_count(y_proc):  # y_proc: (N,3) in PROCESSED (log-count) space
    y2 = y_proc.copy()
    y2[:,0] = np.expm1(y2[:,0])
    return y2

def fit_y_scaler(y_proc):  # mean/std per target on TRAIN (after log1p for count)
    mu = y_proc.mean(axis=0); sd = y_proc.std(axis=0)
    sd = np.where(sd<1e-8, 1.0, sd)
    return mu, sd

def norm_y(y_proc, mu, sd): return (y_proc - mu) / sd
def denorm_y(y_norm, mu, sd): return y_norm * sd + mu

def reg_metrics_all(y_true_orig, y_pred_orig):
    # returns per-target and averaged metrics in ORIGINAL space (count already inverted)
    names = ["count","mean","max"]
    per = {}
    for j, name in enumerate(names):
        per[f"rmse_{name}"] = rmse(y_true_orig[:,j], y_pred_orig[:,j])
        per[f"mae_{name}"]  = mae (y_true_orig[:,j], y_pred_orig[:,j])
        per[f"r2_{name}"]   = r2  (y_true_orig[:,j], y_pred_orig[:,j])
    per["rmse_avg"] = np.mean([per["rmse_count"], per["rmse_mean"], per["rmse_max"]])
    per["mae_avg"]  = np.mean([per["mae_count"],  per["mae_mean"],  per["mae_max"]])
    per["r2_avg"]   = np.mean([per["r2_count"],   per["r2_mean"],   per["r2_max"]])
    return per

def scoring_value_reg(y_true_norm, y_pred_norm, mu, sd):
    # compute mean NEGATIVE RMSE (maximize) in original space
    y_true_proc = denorm_y(y_true_norm, mu, sd)
    y_pred_proc = denorm_y(y_pred_norm, mu, sd)
    y_true_orig = invert_log1p_count(y_true_proc)
    y_pred_orig = invert_log1p_count(y_pred_proc)
    m = reg_metrics_all(y_true_orig, y_pred_orig)
    return -m["rmse_avg"]

# ----------------------- Build master X blocks & Y -----------------------
X_blocks_all = build_numpy_blocks_strict(df.drop(columns=list(FORBID)), GROUPS)
# The columns that went into the 'num' block (exclude one-hots used to form IDs + forbidden)
NUMERIC_FEATURE_NAMES = [
    c for c in df.drop(columns=list(FORBID)).columns
    if not (c.startswith("City_") or c.startswith("County_") or (c.startswith("Weather_") and not c.startswith("Emb_")))
]
assert X_blocks_all["num"].shape[1] == len(NUMERIC_FEATURE_NAMES), \
    f"num block width {X_blocks_all['num'].shape[1]} != names {len(NUMERIC_FEATURE_NAMES)}"

Y_all_raw = df[TARGETS].astype(float).values  # ORIGINAL space

N = len(Y_all_raw)
if HAS_RISK_LEVEL:
    y_split = df["risk_level"].astype(int).values
    outer_pairs = stratified_kfold_indices(y_split, n_splits=OUTER_K, random_state=RANDOM_STATE)
else:
    outer_pairs = kfold_indices(N, n_splits=OUTER_K, random_state=RANDOM_STATE)

Feature Importance Utils

In [ ]:
from copy import deepcopy
from torch.utils.data import DataLoader

def _make_dataset_from_blocks(blocks, y_norm):
    """Build a minimal dataset compatible with your current collate_fn that expects 'num' and optional IDs."""
    class ImpDataset(Dataset):
        def __init__(self, blocks, y_norm):
            self.blocks = blocks
            self.y_norm = y_norm
        def __len__(self): return len(self.y_norm)
        def __getitem__(self, i):
            sample = {"num": self.blocks["num"][i]}
            if "city_id" in self.blocks:   sample["city_id"]   = int(self.blocks["city_id"][i])
            if "county_id" in self.blocks: sample["county_id"] = int(self.blocks["county_id"][i])
            if "wcat_id" in self.blocks:   sample["wcat_id"]   = int(self.blocks["wcat_id"][i])
            return sample, self.y_norm[i]
    return ImpDataset(blocks, y_norm)

@torch.no_grad()
def _score_model_reg(model, blocks, y_norm, mu, sd, collate_fn, device, batch_size=1024):
    ds  = _make_dataset_from_blocks(blocks, y_norm)
    dl  = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
    y_true_norm, y_pred_norm = eval_one_norm(model, dl)
    # use your existing scorer -> negative RMSE (higher is better)
    return scoring_value_reg(y_true_norm, y_pred_norm, mu, sd)

def permutation_importance(
    model, X_blocks, Y_norm, mu, sd, collate_fn,
    block_name=None, feature_names=None, n_repeats=1, device="cuda", seed=42
):
    """
    If block_name is None -> block-level importance over available keys: ['num','city_id','county_id','wcat_id'] (those that exist).
    If block_name == 'num' and feature_names is provided -> per-feature importance within the numeric block.
    Returns (base_score, importance_df)
    """
    rng = np.random.RandomState(seed)

    # ----- baseline -----
    base = _score_model_reg(model, X_blocks, Y_norm, mu, sd, collate_fn, device)

    # ----- block-level mode -----
    if block_name is None:
        keys = [k for k in ["num","city_id","county_id","wcat_id"] if k in X_blocks]
        rows = []
        for k in keys:
            worst = []
            for _ in range(n_repeats):
                Xp = deepcopy(X_blocks)
                arr = Xp[k]
                if arr.ndim == 1:  # IDs
                    rng.shuffle(arr)
                else:               # full matrix -> shuffle rows jointly
                    idx = rng.permutation(arr.shape[0])
                    Xp[k] = arr[idx]
                score = _score_model_reg(model, Xp, Y_norm, mu, sd, collate_fn, device)
                worst.append(base - score)  # drop in score = importance
            rows.append((k, float(np.mean(worst))))
        imp_df = pd.DataFrame(rows, columns=["block","importance"]).sort_values("importance", ascending=False)
        return base, imp_df

    # ----- per-feature mode: only for 'num' -----
    if block_name == "num":
        assert feature_names is not None and len(feature_names) == X_blocks["num"].shape[1], \
            "Provide feature_names for the 'num' block with correct length."
        rows = []
        for j, name in enumerate(feature_names):
            drops = []
            for _ in range(n_repeats):
                Xp = deepcopy(X_blocks)
                col = Xp["num"][:, j].copy()
                rng.shuffle(col)
                Xp["num"][:, j] = col
                score = _score_model_reg(model, Xp, Y_norm, mu, sd, collate_fn, device)
                drops.append(base - score)
            rows.append((name, float(np.mean(drops))))
        imp_df = pd.DataFrame(rows, columns=["feature","importance"]).sort_values("importance", ascending=False)
        return base, imp_df

    raise ValueError(f"Unsupported block_name='{block_name}' for permutation_importance()")


###Outer loop

hyperparameter tuing 1

In [ ]:
# Hyper Parameter Tuning
param_grid = {
    "d_token":  [144, 160, 192],
    "n_layers": [1],
    "lr":       [1.5e-3, 2e-3, 1e-3],
    "dropout":  [0.1],
    "n_heads":  [8],  # adjust if include d_token not divisible by 8
    "weight_decay": [1e-6],
}

def param_grid_iter(grid: dict):
    keys = list(grid.keys()); vals = [grid[k] for k in keys]
    for combo in product(*vals):
        yield dict(zip(keys, combo))

# ----------------------- Outer loop -----------------------
outer_results = []
best_params_all = []
# Collectors for importance across folds
imp_blocks_all = []
imp_weather_all = []
saliency_all = []   # Grad×Input (example for 'weather')


def make_X(idx):
    return {k: (v[idx].copy() if isinstance(v, np.ndarray) else v) for k,v in X_blocks_all.items()}

print("Starting nested CV (multi-task regression)...")
for fold_id, (tr_idx, te_idx) in enumerate(outer_pairs, start=1):
    if VERBOSE: print(f"\n==== Outer fold {fold_id}/{OUTER_K} ====")

    # ----- Build X blocks (train/test) & impute continuous blocks with train medians -----
    X_tr = make_X(tr_idx); X_te = make_X(te_idx)
    def impute_block(arr, med=None):
        if arr.shape[1]==0: return arr
        if med is None: med = fit_medians(arr)
        return apply_medians(arr, med)

    med_num = fit_medians(X_tr["num"]) if X_tr["num"].shape[1] else None
    def impute_num(arr, med=None):
        if arr.shape[1] == 0: return arr
        if med is None: med = fit_medians(arr)
        return apply_medians(arr, med)

    X_tr["num"] = impute_num(X_tr["num"], med_num)
    X_te["num"] = impute_num(X_te["num"], med_num)

    # ----- Build Y (train/test) -----
    Y_tr_raw = Y_all_raw[tr_idx]     # ORIGINAL
    Y_te_raw = Y_all_raw[te_idx]

    # Apply log1p to count target for training stability -> "processed" space
    Y_tr_proc = apply_log1p_count(Y_tr_raw)
    Y_te_proc = apply_log1p_count(Y_te_raw)

    # Standardize per target using OUTER-TRAIN only
    mu, sd = fit_y_scaler(Y_tr_proc)
    Y_tr_norm = norm_y(Y_tr_proc, mu, sd)
    Y_te_norm = norm_y(Y_te_proc, mu, sd)   # used only to pack datasets consistently (not for metrics)

    # ----- Inner CV splits on y_split[tr_idx] or plain KFold -----
    if HAS_RISK_LEVEL:
        inner_pairs = stratified_kfold_indices(df["risk_level"].values[tr_idx], n_splits=INNER_K, random_state=RANDOM_STATE+fold_id)
    else:
        inner_pairs = kfold_indices(len(tr_idx), n_splits=INNER_K, random_state=RANDOM_STATE+fold_id)

    best_score, best_p = -1e9, None

    for p in param_grid_iter(param_grid):
        inner_scores = []
        for in_tr_ind, in_val_ind in inner_pairs:
            # slice X and Y (normalized) for inner split
            in_tr_blocks = {k: v[in_tr_ind] if isinstance(v, np.ndarray) else v for k,v in X_tr.items()}
            in_val_blocks= {k: v[in_val_ind] if isinstance(v, np.ndarray) else v for k,v in X_tr.items()}
            y_in_tr = Y_tr_norm[in_tr_ind]; y_in_val = Y_tr_norm[in_val_ind]

            tr_ds = TabDataset(in_tr_blocks, y_in_tr); va_ds = TabDataset(in_val_blocks, y_in_val)
            tr_loader = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, collate_fn=collate_fn)
            va_loader = DataLoader(va_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)

            model = FTTransformer(
                      n_num=X_tr["num"].shape[1],
                      d_token=p["d_token"], n_layers=p["n_layers"], n_heads=p["n_heads"], dropout=p["dropout"],
                      K_city=X_tr.get("city_oh_K",0), K_county=X_tr.get("county_oh_K",0), K_wcat=X_tr.get("wcat_oh_K",0),
                      n_outputs=3
                  ).to(device)



            optimizer = torch.optim.AdamW(model.parameters(), lr=p["lr"], weight_decay=p["weight_decay"])
            criterion = nn.MSELoss()

            best_val = -1e9; no_improve = 0
            for epoch in range(1, MAX_EPOCHS+1):
                _ = train_one(model, tr_loader, optimizer, criterion)
                y_val_true_norm, y_val_pred_norm = eval_one_norm(model, va_loader)
                score = scoring_value_reg(y_val_true_norm, y_val_pred_norm, mu, sd)  # uses ORIGINAL-scale RMSE (avg)

                if score > best_val + 1e-6:
                    best_val = score; no_improve = 0
                    best_state = {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}
                else:
                    no_improve += 1
                if no_improve >= PATIENCE: break

            model.load_state_dict(best_state)
            # final inner val score
            y_val_true_norm, y_val_pred_norm = eval_one_norm(model, va_loader)
            inner_scores.append(scoring_value_reg(y_val_true_norm, y_val_pred_norm, mu, sd))

        mean_inner = float(np.mean(inner_scores))
        if VERBOSE: print(f"  Params {p} -> inner mean {-mean_inner:.4f} RMSE (lower is better)")
        if mean_inner > best_score:
            best_score, best_p = mean_inner, p

    best_params_all.append(best_p)
    if VERBOSE: print(f"Best inner (mean RMSE): {-best_score:.4f} with {best_p}")

    # ----- Retrain on FULL outer-train with best params (with small ES split) -----
    tr_ds_full = TabDataset(X_tr, Y_tr_norm); te_ds = TabDataset(X_te, Y_te_norm)
    tr_loader_full = DataLoader(tr_ds_full, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, collate_fn=collate_fn)
    te_loader      = DataLoader(te_ds,      batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)

    model = FTTransformer(
        n_num=X_tr["num"].shape[1],
        d_token=best_p["d_token"], n_layers=best_p["n_layers"], n_heads=best_p["n_heads"], dropout=best_p["dropout"],
        K_city=X_tr.get("city_oh_K",0), K_county=X_tr.get("county_oh_K",0), K_wcat=X_tr.get("wcat_oh_K",0),
        n_outputs=3
    ).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=best_p["lr"], weight_decay=best_p["weight_decay"])
    criterion = nn.MSELoss()

    idx_all = np.arange(len(tr_idx)); np.random.shuffle(idx_all)
    cut = max(1, int(0.1*len(idx_all)))
    es_val = idx_all[:cut]; es_tr = idx_all[cut:]
    es_tr_loader = DataLoader(Subset(tr_ds_full, es_tr), batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
    es_va_loader = DataLoader(Subset(tr_ds_full, es_val), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    best_val = -1e9; no_improve = 0
    for epoch in range(1, MAX_EPOCHS+1):
        _ = train_one(model, es_tr_loader, optimizer, criterion)
        y_val_true_norm, y_val_pred_norm = eval_one_norm(model, es_va_loader)
        score = scoring_value_reg(y_val_true_norm, y_val_pred_norm, mu, sd)
        if score > best_val + 1e-6:
            best_val = score; no_improve = 0
            best_state = {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}
        else:
            no_improve += 1
        if no_improve >= PATIENCE: break
    model.load_state_dict(best_state)

    # ----- Evaluate on OUTER-TEST in ORIGINAL space -----
    y_te_true_norm, y_te_pred_norm = eval_one_norm(model, te_loader)
    y_te_true_proc = denorm_y(y_te_true_norm, mu, sd)
    y_te_pred_proc = denorm_y(y_te_pred_norm, mu, sd)
    y_te_true_orig = invert_log1p_count(y_te_true_proc)
    y_te_pred_orig = invert_log1p_count(y_te_pred_proc)

    m = reg_metrics_all(y_te_true_orig, y_te_pred_orig)
    m["fold"] = fold_id
    outer_results.append(m)

    # --- Feature importance on OUTER-TEST for this fold ---
    # (A) Block-level permutation importance over ['num', 'city_id', 'county_id', 'wcat_id'] (whatever exists)
    base_metrics_imp, imp_blocks = permutation_importance(
        model, X_te, Y_te_norm, mu, sd, collate_fn,
        block_name=None, device=device
    )
    imp_blocks["fold"] = fold_id
    imp_blocks_all.append(imp_blocks)

    # (B) Per-feature permutation inside the numeric block
    if X_te["num"].shape[1] > 0:
        _, imp_num = permutation_importance(
            model, X_te, Y_te_norm, mu, sd, collate_fn,
            block_name="num", feature_names=NUMERIC_FEATURE_NAMES, device=device
        )
        imp_num["fold"] = fold_id
        imp_weather_all.append(imp_num)   # or rename this collector to imp_num_all


Starting nested CV (multi-task regression)...

==== Outer fold 1/10 ====


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2446 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2509 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2388 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2410 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2498 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2417 RMSE (lower is better)
  Params {'d_token': 192, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2526 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2481 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2393 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2557 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2505 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2391 RMSE (lower is better)
  Params {'d_token': 192, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight

...

  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2493 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2548 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2393 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2492 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2465 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2413 RMSE (lower is better)
  Params {'d_token': 192, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2451 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2477 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2365 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2458 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2554 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2476 RMSE (lower is better)
  Params {'d_token': 192, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2441 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2426 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2393 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2439 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.002, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2538 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2383 RMSE (lower is better)
  Params {'d_token': 192, 'n_layers': 1, 'lr': 0.0015, 'dropout': 0.1, 'n_heads': 8, 'weight

Report 1

In [ ]:
res_df = pd.DataFrame(outer_results).set_index("fold").sort_index()
mean = res_df.mean(numeric_only=True); se = res_df.std(ddof=1, numeric_only=True) / np.sqrt(len(res_df))

print("\nResults by fold (REGRESSION):")
print(res_df[[c for c in res_df.columns if c.startswith("rmse_") or c.startswith("r2_")]])

print("\nMean ± SE (original scale):")
for k in ["rmse_count","rmse_mean","rmse_max","rmse_avg","mae_count","mae_mean","mae_max","mae_avg","r2_count","r2_mean","r2_max","r2_avg"]:
    print(f"{k:>10}: {mean.get(k,np.nan):.4f} ± {se.get(k,np.nan):.4f}")


hp_df = pd.DataFrame(best_params_all)
print("\nMost selected hyperparameters:")
for col in hp_df.columns:
    try:
        print(f"- {col}: {Counter(hp_df[col]).most_common(1)[0][0]}")
    except Exception:
        pass


Results by fold (REGRESSION):
      rmse_count  r2_count  rmse_mean   r2_mean  rmse_max    r2_max  rmse_avg  \
fold                                                                            
1       0.045984  0.820209   0.064566  0.442462  0.595408  0.485413  0.235319   
2       0.049296  0.791592   0.069477  0.467236  0.635808  0.463234  0.251527   
3       0.046567  0.818129   0.060587  0.478808  0.588562  0.491241  0.231905   
4       0.046632  0.810017   0.066050  0.540523  0.622165  0.503153  0.244949   
5       0.048003  0.798642   0.063633  0.484587  0.621159  0.464156  0.244265   
6       0.044544  0.829398   0.060543  0.543613  0.609585  0.490584  0.238224   
7       0.049904  0.780022   0.064174  0.480672  0.630515  0.419564  0.248198   
8       0.046207  0.819106   0.061453  0.533168  0.570610  0.529049  0.226090   
9       0.048198  0.790386   0.068018  0.434531  0.618926  0.444314  0.245047   
10      0.049965  0.793837   0.067004  0.546350  0.587581  0.515689  0.234850 

In [ ]:
# ---- Aggregate importance over folds ----
if len(imp_blocks_all):
    imp_blocks_df = pd.concat(imp_blocks_all, ignore_index=True)
    # columns are: ['block', 'importance', 'fold']
    print("\nBlock permutation importance (ΔRMSE, mean over folds — higher = more important):")
    print(
        imp_blocks_df.groupby("block")["importance"]
        .mean()
        .sort_values(ascending=False)
    )

if len(imp_weather_all):
    # This list actually holds per-feature importance for the numeric block.
    # If you renamed it to imp_num_all, change the variable name accordingly.
    imp_num_df = pd.concat(imp_weather_all, ignore_index=True)
    # columns are: ['feature', 'importance', 'fold']
    print("\nTop NUMERIC features by ΔRMSE (mean over folds):")
    print(
        imp_num_df.groupby("feature")["importance"]
        .mean()
        .sort_values(ascending=False)
        .head(20)
    )

# If you collected Grad×Input saliency per feature into saliency_all (a list of Series),
# this still works. Just make sure the Series index are numeric feature names.
if len(saliency_all):
    saliency_mat = pd.concat(saliency_all, axis=1)  # columns = folds
    print("\nGrad×Input saliency (NUM) — mean over folds:")
    print(saliency_mat.mean(axis=1).sort_values(ascending=False).head(20))



Block permutation importance (ΔRMSE, mean over folds — higher = more important):
block
num          0.188599
county_id    0.000568
city_id      0.000301
wcat_id      0.000175
Name: importance, dtype: float64

Top NUMERIC features by ΔRMSE (mean over folds):
feature
Emb_PCA_0                       0.073792
Emb_PCA_4                       0.055127
Emb_PCA_6                       0.028085
Emb_PCA_2                       0.010743
Emb_PCA_3                       0.006116
Emb_PCA_5                       0.005474
Emb_PCA_8                       0.004856
Emb_PCA_1                       0.004360
Bump                            0.003013
Amenity                         0.001962
Emb_PCA_26                      0.001876
Astronomical_Twilight_binary    0.001659
Emb_PCA_14                      0.001576
Start_Month_sin                 0.001528
Emb_PCA_7                       0.001523
Start_Hour_sin                  0.001374
Wind_Direction_cos              0.001357
Emb_PCA_43                      0.00

hyperparameter tuning 2

In [ ]:
# Hyper Parameter Tuning
param_grid = {
    "d_token":  [112, 128, 144],
    "n_layers": [2, 3, 4],
    "lr":       [1e-3],
    "dropout":  [0.1],
    "n_heads":  [8],  # adjust if include d_token not divisible by 8
    "weight_decay": [1e-6],
}

def param_grid_iter(grid: dict):
    keys = list(grid.keys()); vals = [grid[k] for k in keys]
    for combo in product(*vals):
        yield dict(zip(keys, combo))

# ----------------------- Outer loop -----------------------
outer_results = []
best_params_all = []
# Collectors for importance across folds
imp_blocks_all = []
imp_weather_all = []
saliency_all = []   # Grad×Input (example for 'weather')


def make_X(idx):
    return {k: (v[idx].copy() if isinstance(v, np.ndarray) else v) for k,v in X_blocks_all.items()}

print("Starting nested CV (multi-task regression)...")
for fold_id, (tr_idx, te_idx) in enumerate(outer_pairs, start=1):
    if VERBOSE: print(f"\n==== Outer fold {fold_id}/{OUTER_K} ====")

    # ----- Build X blocks (train/test) & impute continuous blocks with train medians -----
    X_tr = make_X(tr_idx); X_te = make_X(te_idx)
    def impute_block(arr, med=None):
        if arr.shape[1]==0: return arr
        if med is None: med = fit_medians(arr)
        return apply_medians(arr, med)

    med_num = fit_medians(X_tr["num"]) if X_tr["num"].shape[1] else None
    def impute_num(arr, med=None):
        if arr.shape[1] == 0: return arr
        if med is None: med = fit_medians(arr)
        return apply_medians(arr, med)

    X_tr["num"] = impute_num(X_tr["num"], med_num)
    X_te["num"] = impute_num(X_te["num"], med_num)

    # ----- Build Y (train/test) -----
    Y_tr_raw = Y_all_raw[tr_idx]     # ORIGINAL
    Y_te_raw = Y_all_raw[te_idx]

    # Apply log1p to count target for training stability -> "processed" space
    Y_tr_proc = apply_log1p_count(Y_tr_raw)
    Y_te_proc = apply_log1p_count(Y_te_raw)

    # Standardize per target using OUTER-TRAIN only
    mu, sd = fit_y_scaler(Y_tr_proc)
    Y_tr_norm = norm_y(Y_tr_proc, mu, sd)
    Y_te_norm = norm_y(Y_te_proc, mu, sd)   # used only to pack datasets consistently (not for metrics)

    # ----- Inner CV splits on y_split[tr_idx] or plain KFold -----
    if HAS_RISK_LEVEL:
        inner_pairs = stratified_kfold_indices(df["risk_level"].values[tr_idx], n_splits=INNER_K, random_state=RANDOM_STATE+fold_id)
    else:
        inner_pairs = kfold_indices(len(tr_idx), n_splits=INNER_K, random_state=RANDOM_STATE+fold_id)

    best_score, best_p = -1e9, None

    for p in param_grid_iter(param_grid):
        inner_scores = []
        for in_tr_ind, in_val_ind in inner_pairs:
            # slice X and Y (normalized) for inner split
            in_tr_blocks = {k: v[in_tr_ind] if isinstance(v, np.ndarray) else v for k,v in X_tr.items()}
            in_val_blocks= {k: v[in_val_ind] if isinstance(v, np.ndarray) else v for k,v in X_tr.items()}
            y_in_tr = Y_tr_norm[in_tr_ind]; y_in_val = Y_tr_norm[in_val_ind]

            tr_ds = TabDataset(in_tr_blocks, y_in_tr); va_ds = TabDataset(in_val_blocks, y_in_val)
            tr_loader = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, collate_fn=collate_fn)
            va_loader = DataLoader(va_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)

            model = FTTransformer(
                      n_num=X_tr["num"].shape[1],
                      d_token=p["d_token"], n_layers=p["n_layers"], n_heads=p["n_heads"], dropout=p["dropout"],
                      K_city=X_tr.get("city_oh_K",0), K_county=X_tr.get("county_oh_K",0), K_wcat=X_tr.get("wcat_oh_K",0),
                      n_outputs=3
                  ).to(device)



            optimizer = torch.optim.AdamW(model.parameters(), lr=p["lr"], weight_decay=p["weight_decay"])
            criterion = nn.MSELoss()

            best_val = -1e9; no_improve = 0
            for epoch in range(1, MAX_EPOCHS+1):
                _ = train_one(model, tr_loader, optimizer, criterion)
                y_val_true_norm, y_val_pred_norm = eval_one_norm(model, va_loader)
                score = scoring_value_reg(y_val_true_norm, y_val_pred_norm, mu, sd)  # uses ORIGINAL-scale RMSE (avg)

                if score > best_val + 1e-6:
                    best_val = score; no_improve = 0
                    best_state = {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}
                else:
                    no_improve += 1
                if no_improve >= PATIENCE: break

            model.load_state_dict(best_state)
            # final inner val score
            y_val_true_norm, y_val_pred_norm = eval_one_norm(model, va_loader)
            inner_scores.append(scoring_value_reg(y_val_true_norm, y_val_pred_norm, mu, sd))

        mean_inner = float(np.mean(inner_scores))
        if VERBOSE: print(f"  Params {p} -> inner mean {-mean_inner:.4f} RMSE (lower is better)")
        if mean_inner > best_score:
            best_score, best_p = mean_inner, p

    best_params_all.append(best_p)
    if VERBOSE: print(f"Best inner (mean RMSE): {-best_score:.4f} with {best_p}")

    # ----- Retrain on FULL outer-train with best params (with small ES split) -----
    tr_ds_full = TabDataset(X_tr, Y_tr_norm); te_ds = TabDataset(X_te, Y_te_norm)
    tr_loader_full = DataLoader(tr_ds_full, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0, collate_fn=collate_fn)
    te_loader      = DataLoader(te_ds,      batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=collate_fn)

    model = FTTransformer(
        n_num=X_tr["num"].shape[1],
        d_token=best_p["d_token"], n_layers=best_p["n_layers"], n_heads=best_p["n_heads"], dropout=best_p["dropout"],
        K_city=X_tr.get("city_oh_K",0), K_county=X_tr.get("county_oh_K",0), K_wcat=X_tr.get("wcat_oh_K",0),
        n_outputs=3
    ).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=best_p["lr"], weight_decay=best_p["weight_decay"])
    criterion = nn.MSELoss()

    idx_all = np.arange(len(tr_idx)); np.random.shuffle(idx_all)
    cut = max(1, int(0.1*len(idx_all)))
    es_val = idx_all[:cut]; es_tr = idx_all[cut:]
    es_tr_loader = DataLoader(Subset(tr_ds_full, es_tr), batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
    es_va_loader = DataLoader(Subset(tr_ds_full, es_val), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    best_val = -1e9; no_improve = 0
    for epoch in range(1, MAX_EPOCHS+1):
        _ = train_one(model, es_tr_loader, optimizer, criterion)
        y_val_true_norm, y_val_pred_norm = eval_one_norm(model, es_va_loader)
        score = scoring_value_reg(y_val_true_norm, y_val_pred_norm, mu, sd)
        if score > best_val + 1e-6:
            best_val = score; no_improve = 0
            best_state = {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}
        else:
            no_improve += 1
        if no_improve >= PATIENCE: break
    model.load_state_dict(best_state)

    # ----- Evaluate on OUTER-TEST in ORIGINAL space -----
    y_te_true_norm, y_te_pred_norm = eval_one_norm(model, te_loader)
    y_te_true_proc = denorm_y(y_te_true_norm, mu, sd)
    y_te_pred_proc = denorm_y(y_te_pred_norm, mu, sd)
    y_te_true_orig = invert_log1p_count(y_te_true_proc)
    y_te_pred_orig = invert_log1p_count(y_te_pred_proc)

    m = reg_metrics_all(y_te_true_orig, y_te_pred_orig)
    m["fold"] = fold_id
    outer_results.append(m)

    # --- Feature importance on OUTER-TEST for this fold ---
    # (A) Block-level permutation importance over ['num', 'city_id', 'county_id', 'wcat_id'] (whatever exists)
    base_metrics_imp, imp_blocks = permutation_importance(
        model, X_te, Y_te_norm, mu, sd, collate_fn,
        block_name=None, device=device
    )
    imp_blocks["fold"] = fold_id
    imp_blocks_all.append(imp_blocks)

    # (B) Per-feature permutation inside the numeric block
    if X_te["num"].shape[1] > 0:
        _, imp_num = permutation_importance(
            model, X_te, Y_te_norm, mu, sd, collate_fn,
            block_name="num", feature_names=NUMERIC_FEATURE_NAMES, device=device
        )
        imp_num["fold"] = fold_id
        imp_weather_all.append(imp_num)   # or rename this collector to imp_num_all


Starting nested CV (multi-task regression)...

==== Outer fold 1/10 ====


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 128, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2402 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2383 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2333 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2385 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2381 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2357 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_de

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 128, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2336 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2357 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2334 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2449 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2355 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2356 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_de

...

  Params {'d_token': 128, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2413 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2368 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2350 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2455 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2431 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2393 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_de

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 128, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2449 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2375 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2371 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2385 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2389 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2389 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_de

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  Params {'d_token': 128, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2398 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2351 RMSE (lower is better)
  Params {'d_token': 128, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2378 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2419 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 2, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2372 RMSE (lower is better)
  Params {'d_token': 144, 'n_layers': 3, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_decay': 1e-06} -> inner mean 0.2366 RMSE (lower is better)
  Params {'d_token': 160, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.1, 'n_heads': 8, 'weight_de

report 2 (per target evaluation)

In [ ]:
res_df = pd.DataFrame(outer_results).set_index("fold").sort_index()
mean = res_df.mean(numeric_only=True); se = res_df.std(ddof=1, numeric_only=True) / np.sqrt(len(res_df))

for name in ["count", "mean", "max"]:
    print(f"\n=== Per-target metrics for severity_{name} (outer 10-fold) ===")
    cols = [f"rmse_{name}", f"mae_{name}", f"r2_{name}"]
    print(res_df[cols])

    print("Mean ± SE:")
    print(f"RMSE: {mean[f'rmse_{name}']:.4f} ± {se[f'rmse_{name}']:.4f}")
    print(f"MAE : {mean[f'mae_{name}']:.4f} ± {se[f'mae_{name}']:.4f}")
    print(f"R²  : {mean[f'r2_{name}']:.4f} ± {se[f'r2_{name}']:.4f}")

hp_df = pd.DataFrame(best_params_all)
print("\nMost selected hyperparameters:")
for col in hp_df.columns:
    try:
        print(f"- {col}: {Counter(hp_df[col]).most_common(1)[0][0]}")
    except Exception:
        pass


=== Per-target metrics for severity_count (outer 10-fold) ===
      rmse_count  mae_count  r2_count
fold                                 
1       0.046371   0.035472  0.817168
2       0.045154   0.034809  0.825144
3       0.050471   0.039288  0.786355
4       0.044601   0.034290  0.826208
5       0.047945   0.037719  0.799128
6       0.044266   0.035037  0.831521
7       0.046558   0.035449  0.808535
8       0.043475   0.032186  0.839865
9       0.043885   0.034321  0.826222
10      0.045750   0.034721  0.827158
Mean ± SE:
RMSE: 0.0458 ± 0.0007
MAE : 0.0353 ± 0.0006
R²  : 0.8187 ± 0.0051

=== Per-target metrics for severity_mean (outer 10-fold) ===
      rmse_mean  mae_mean   r2_mean
fold                               
1      0.064777  0.027771  0.438807
2      0.065997  0.028497  0.519267
3      0.059477  0.026402  0.497732
4      0.065154  0.029949  0.552897
5      0.061635  0.028521  0.516438
6      0.060576  0.025767  0.543122
7      0.062115  0.027988  0.513465
8      0.059364  0

In [ ]:
# ---- Aggregate importance over folds ----
if len(imp_blocks_all):
    imp_blocks_df = pd.concat(imp_blocks_all, ignore_index=True)
    # columns are: ['block', 'importance', 'fold']
    print("\nBlock permutation importance (ΔRMSE, mean over folds — higher = more important):")
    print(
        imp_blocks_df.groupby("block")["importance"]
        .mean()
        .sort_values(ascending=False)
    )

if len(imp_weather_all):
    # This list actually holds per-feature importance for the numeric block.
    # If you renamed it to imp_num_all, change the variable name accordingly.
    imp_num_df = pd.concat(imp_weather_all, ignore_index=True)
    # columns are: ['feature', 'importance', 'fold']
    print("\nTop NUMERIC features by ΔRMSE (mean over folds):")
    print(
        imp_num_df.groupby("feature")["importance"]
        .mean()
        .sort_values(ascending=False)
        .head(20)
    )

# If you collected Grad×Input saliency per feature into saliency_all (a list of Series),
# this still works. Just make sure the Series index are numeric feature names.
if len(saliency_all):
    saliency_mat = pd.concat(saliency_all, axis=1)  # columns = folds
    print("\nGrad×Input saliency (NUM) — mean over folds:")
    print(saliency_mat.mean(axis=1).sort_values(ascending=False).head(20))



Block permutation importance (ΔRMSE, mean over folds — higher = more important):
block
num          0.201438
county_id    0.000316
wcat_id      0.000110
city_id     -0.000068
Name: importance, dtype: float64

Top NUMERIC features by ΔRMSE (mean over folds):
feature
Emb_PCA_0                       0.086090
Emb_PCA_4                       0.067645
Emb_PCA_6                       0.030021
Emb_PCA_2                       0.010963
Emb_PCA_3                       0.007446
Emb_PCA_8                       0.005987
Emb_PCA_5                       0.004327
Emb_PCA_1                       0.004063
Emb_PCA_10                      0.002963
Emb_PCA_31                      0.002634
Emb_PCA_14                      0.002264
Emb_PCA_23                      0.002112
Bump                            0.002097
Astronomical_Twilight_binary    0.002085
Emb_PCA_7                       0.001997
Amenity                         0.001978
Emb_PCA_22                      0.001971
Emb_PCA_24                      0.00